In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np


In [2]:
data_in = np.loadtxt(fname='./test-score.csv', delimiter=',')
x_train = torch.from_numpy(data_in[:, 0:3]).float()             # (N, 3)
y_train = torch.from_numpy(data_in[:, 3]).unsqueeze(1).float()  # (N, 1)

In [3]:
class LinearRegressionModel(nn.Module):
    #__init__(): 객체가 갖는 속성값을 초기화하는 역할 -> 객체가 생성될 때 자동으로 호출
    # super()호출 >>>> 오버라이딩,  nn.Module 클래스의 속성들을 가지고 초기화 
    def __init__(self, W, b):
        super().__init__()
        self.linear = nn.Linear(3, 1)
        self.linear.bias = nn.Parameter(b, requires_grad=True)
        self.linear.weight = nn.Parameter(W.T, requires_grad=True)

    #forward() :모델이 학습데이터를 입력받아서 forward 연산을 진행 -> 객체를 데이터와 함께 호출하면 자동으로 실행
    # forward 연산: H(x)  식에 입력 x로부터 예측된 y를 얻는 것
    
    def forward(self, x):
        return self.linear(x)

In [4]:
W = torch.zeros((3, 1))
b = torch.zeros(1)
# model.parameters([W, b])
model = LinearRegressionModel(W, b)
# model.parameters()


In [5]:
# Set up the optimizer
optimizer = optim.SGD(model.parameters(), lr=1e-5)
nb_epochs = 100000

In [6]:
model.train()

for epoch in range(nb_epochs + 1):
    # Compute H(x)
    prediction = model(x_train)

    # Compute the cost with MSE
    cost = F.mse_loss(prediction, y_train)
    # Update H(x)
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    # print epoch and cost for every 20 epochs
    
    if epoch % 10000 == 0:
        params = list(model.parameters())
        W = params[0].detach().numpy()
        b = params[1].item()
        print('Epoch {:4d}/{} W: {}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, tuple(*W), b, cost.item()
        ))



Epoch    0/100000 W: (0.26151597, 0.26294398, 0.26934156), b: 0.003 Cost: 26811.960938
Epoch 10000/100000 W: (0.38032684, 0.5210467, 1.112572), b: -0.005 Cost: 6.110940
Epoch 20000/100000 W: (0.35797518, 0.5294029, 1.1263276), b: -0.021 Cost: 6.096858
Epoch 30000/100000 W: (0.3561641, 0.53038293, 1.12733), b: -0.038 Cost: 6.094028
Epoch 40000/100000 W: (0.3560846, 0.53047836, 1.1275129), b: -0.055 Cost: 6.091289
Epoch 50000/100000 W: (0.3560721, 0.53054136, 1.127662), b: -0.071 Cost: 6.088572
Epoch 60000/100000 W: (0.35606483, 0.5305922, 1.127817), b: -0.087 Cost: 6.085886
Epoch 70000/100000 W: (0.35606423, 0.5306378, 1.1279699), b: -0.104 Cost: 6.083202
Epoch 80000/100000 W: (0.3560635, 0.5306838, 1.1281216), b: -0.120 Cost: 6.080553
Epoch 90000/100000 W: (0.356064, 0.53072816, 1.1282735), b: -0.136 Cost: 6.077927
Epoch 100000/100000 W: (0.35606432, 0.530774, 1.1284227), b: -0.152 Cost: 6.075311


In [7]:
model.eval()
# Put the test input
new_var = torch.FloatTensor([[73, 80, 75]])
# Get the prediction value.
pred_y = model(new_var)
print(pred_y)


tensor([[152.9338]], grad_fn=<AddmmBackward0>)
